In [1]:
import numpy as np
from models.layers import layers
from models.layers.initializers import he_normalizer

In [40]:
import datetime
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
import cv2
import random
#import skimage.io as io
#import skimage.transform as trans

import tensorflow as tf
import logging
from tensorflow.keras.layers import  Input, Conv2D, MaxPooling2D,Dropout,UpSampling2D,concatenate,BatchNormalization,Activation,add
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import glob
import pickle
# In[1]:
import tensorflow.keras as keras
import tensorflow.keras.backend as K

In [3]:
import torch
import torch.nn as nn

In [4]:
classes = 2
channels = 6
# x = np.random.randn(10, 6, 128, 128)
x = np.random.randn(1, 1, 3, 3)
x = (x - x.min(axis=3, keepdims=True)) / (x.max(axis=3, keepdims=True) - x.min(axis=3, keepdims=True))

In [5]:
torch_conv = nn.Conv2d(1, 1, 3, padding="same")
tf_conv = tf.keras.layers.Conv2D(1, 3, activation=None, padding="same")

In [6]:
tf_x = tf.convert_to_tensor(x.transpose(0, 2, 3, 1))
torch_x = torch.Tensor(x)

2023-06-20 02:19:59.353926: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-06-20 02:19:59.353950: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: tuf
2023-06-20 02:19:59.353957: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: tuf
2023-06-20 02:19:59.353987: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2023-06-20 02:19:59.354001: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.108.3
2023-06-20 02:19:59.354004: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.108.3
2023-06-20 02:19:59.354173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operation

In [7]:
torch_y = torch_conv(torch_x)
tf_y = tf_conv(tf_x)

In [8]:
torch_y

tensor([[[[-0.2907, -0.3869, -0.6376],
          [-0.0041, -0.2747, -0.7813],
          [ 0.3725, -0.3935, -0.8194]]]], grad_fn=<ConvolutionBackward0>)

In [9]:
tf_y

<tf.Tensor: shape=(1, 3, 3, 1), dtype=float32, numpy=
array([[[[-0.05410799],
         [ 0.08171681],
         [ 0.38680807]],

        [[ 0.73151803],
         [ 0.8457    ],
         [ 0.7214397 ]],

        [[ 1.1782757 ],
         [ 0.92955893],
         [ 0.694498  ]]]], dtype=float32)>

In [10]:
torch_y.shape, tf_y.shape

(torch.Size([1, 1, 3, 3]), TensorShape([1, 3, 3, 1]))

In [11]:
tf_conv.activation

<function keras.activations.linear(x)>

In [12]:
x = np.random.randn(6, 3, 128, 128)
x = (x - x.min(axis=3, keepdims=True)) / (x.max(axis=3, keepdims=True) - x.min(axis=3, keepdims=True))

In [43]:
seq = nn.Sequential(
    layers.CustomConv2d(3, 64, 5, (1,1), 2, 2),
    layers.CustomConv2d(64, 128, 5, (1,1), 2, 2),
    layers.CustomConv2d(128, 256, 5, (1,1), 2, 2),
    layers.CustomConv2d(256, 512, 5, (1,1), 2, 2, dropout=0.5),
)

In [44]:
torch_x = torch.Tensor(x)
tf_x = tf.convert_to_tensor(x)
tf_x = tf.transpose(tf_x, perm=[0, 2, 3, 1])
torch_x.shape, tf_x.shape

(torch.Size([6, 3, 128, 128]), TensorShape([6, 128, 128, 3]))

In [45]:
torch_y = seq(torch_x)

In [46]:
conv1_1_L = Conv2D(64, 5, strides=(1, 1), padding='same', use_bias=False, kernel_initializer='he_normal')(tf_x)
conv1_1_L = BatchNormalization(axis=3)(conv1_1_L)
conv1_1_L = Activation('relu')(conv1_1_L)
conv1_1_L = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv1_1_L)
#drop1_2_L = Dropout(0.25)(conv1_1_L)

# conv2_x  input size:64
conv2_1_L = Conv2D(128, 5, strides=(1, 1), padding='same', use_bias=False, kernel_initializer='he_normal')(conv1_1_L)
conv2_1_L = BatchNormalization(axis=3)(conv2_1_L)
conv2_1_L = Activation('relu')(conv2_1_L)
conv2_1_L = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv2_1_L)
#drop2_2_L = Dropout(0.5)(conv2_1_L)

# conv3_x  input size:32
conv3_1_L = Conv2D(256, 5, strides=(1, 1), padding='same', use_bias=False, kernel_initializer='he_normal')(conv2_1_L)
conv3_1_L = BatchNormalization(axis=3)(conv3_1_L)
conv3_1_L = Activation('relu')(conv3_1_L)
conv3_1_L = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv3_1_L)
#drop3_2_L = Dropout(0.5)(conv3_1_L)
# conv3_x  input size:16
conv4_1_L = Conv2D(512, 5, strides=(1, 1), padding='same', use_bias=False, kernel_initializer='he_normal')(conv3_1_L)
conv4_1_L = BatchNormalization(axis=3)(conv4_1_L)
conv4_1_L = Activation('relu')(conv4_1_L)
conv4_1_L = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv4_1_L)
drop4_2_L = Dropout(0.5)(conv4_1_L)


In [48]:
drop4_2_L.shape, torch_y.shape

(TensorShape([6, 8, 8, 512]), torch.Size([6, 512, 8, 8]))

In [49]:
c1=32
c1=int(drop4_2_L.shape[3]/8)
c=4
c=drop4_2_L.shape[2]
d=256
d=drop4_2_L.shape[3]
fa=Conv2D(c1,1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop4_2_L)
fb=Conv2D(c1,1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop4_2_L)
fc=Conv2D(d,1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop4_2_L)

In [53]:
c1, c, d, drop4_2_L.shape, fa.shape, fb.shape, fc.shape

(64,
 8,
 512,
 TensorShape([6, 8, 8, 512]),
 TensorShape([6, 8, 8, 64]),
 TensorShape([6, 8, 8, 64]),
 TensorShape([6, 8, 8, 512]))

In [58]:
torch_y.size(), torch_y.size(1) // 8, torch_y.size(3), torch_y.size(1)

(torch.Size([6, 512, 8, 8]), 64, 8, 512)

In [59]:
fa1=tf.reshape(fa,[-1,c*c,c1])
fb1=tf.reshape(fb,[-1,c*c,c1])

In [60]:
fa1.shape, fb1.shape

(TensorShape([6, 64, 64]), TensorShape([6, 64, 64]))

In [62]:
torch_y.view(-1, c1).shape

torch.Size([196608])